# MongoDB Tutorial
## *Prof. Gary L. Pavlis*

## Overview
This notebook is designed as a teaching tutorial on use of the MongoDB database used in MsPASS.  Numerous pedagogic materials exist online for learning MongoDB, but this notebook focuses on key features the author has found useful in seismology research.  It is best used in conjunction with two other sources:
1.  The section of the User's Manual titled "Using MongoDB with MsPASS".
2.  As with most modern IT topics a web search for details of some topics addressed in this tutorial may be helpful if the MsPASS User's Manual doesn't address the topic.

The bulk of this notebook is organized by the keywords of the standard CRUD acronymn of database theory.  CRUD is an abbreviation of Create (save), Read, Update, and Delete.  Section are titled with those keywords and covered in the order defined by CRUD.  Before that, however, it is necessary to review a few basic concepts covered in the section immediately below.

## MongoDB Core Concepts
### Client-server model
MongoDB is a client-server system.  That bit of jargon 
has some important implications:

1.  All database commands issued from python are not executed directly by the python interpreter  Instead instructions are sent to the MongoDB server.   In MsPASS the server is launched inside a container.   Unless you are running this notebook on a cluster with multiple nodes, you can verify the server is running by launching a terminal in the jupyterlab interface and running the command `ps -A`.  You should get output similar to the following that shows the server as the CMD with the name `mongod`:
```
root@b0d79c4cc440:/home/scoped# ps -A
  PID TTY          TIME CMD
    1 ?        00:00:00 tini
    8 ?        00:00:00 start-mspass.sh
   15 ?        00:07:27 dask-scheduler
   21 ?        00:06:47 dask-worker
   22 ?        00:01:44 mongod
   23 ?        00:00:44 jupyter-lab
   34 ?        00:00:00 python3.10
   37 ?        00:10:43 python3.10
  154 ?        00:00:20 python
  364 ?        00:00:01 python
 1010 pts/0    00:00:00 bash
 1036 pts/0    00:00:00 ps
```
2.  All database IO passes through a network data connection on network "port number" 27017.   That is important to know as a fundamental issue because a network communication channel is not the fastest data pipe on most computers.
3.  To communicate with MongoDB, your program must create a connection to the "server".  In the jargon of modern computing you have to create a "client" that will act as your agent to talk to the arrogant MongoDB "server" (the mongod program running in the background).  

With that background, the first thing you will need to do, since mongod is already running in this environment, is to create the "client".    

In [3]:
from mspasspy.db import DBClient
dbclient=DBClient()

A geeky detail worth noting here is that we are using a python class (object) called `DBClient` that is a "subclass" of `pymongo.MongoClient`.  I point that out because all internet sources that are MongoDB introductions will create an instance of `pymongo.MongoClient` instead of the MsPASS extension used above.  An important "extension" DBClient adds is illustrated by the next code box:

In [4]:
db = dbclient.get_database("dbtutorial")

This incantation runs the `get_database` "method" of the class called `DBClient`.   It returns what we call a "database handle" in the User's Manual.   The MsPASS "database handle" is a python class that is itself a subclass  of another pymongo class.  Both have the name `Database`, but the MsPASS version adds a number of extensions for handling of seismic data.   The main ones of interest are readers and writers for seismic data objects, station metadata, and source metadata.  A key point is almost all MsPASS workflows begin with a variation of the combination of the two python code boxes above.   

When you call the `get_database` method as shown above the "handle" is created/constructed and can be accessed for the rest of your python workflow with the symbol you put on the left hand side of the expression (`db` in this example).  That name, of course, can be anything you want it to be, but For all examples in the MsPASS documentation we used `db` as a standard symbol to reduce confusion, but that should be viewed as simply a notation convention not a rule.  

### Documents and Collections
The User's Manual section companion to this tutorial discusses the MongoDB jargon terms `document` and `collection` at length.   I will not repeat that material here, but not from here on I assume you know what those two terms.   If you don't know what these terms mean consult the "Using MongoDB with MsPASS" section of the User's Manual or some other source before proceeding.

## Create
The first letter in the CRUD acronynm is "Create".  For this tutorial some form of "create" is an essential first step to put some kind of data into our tutorial database.   Most tutorials will begin inserting some largely arbitrary data.  Since this tutorial is designed for seismologists it seems more appropriate to work with seismology data.   The box below is a variant of one in the "getting_started" tutorial. It uses obspy's web service module to fetch station metadata for all "B-channels" defined for Earthscope TA stations that operated during the calendar year 2011. Obspy creates a python image of the stationxml downloaded from IRIS they call an `Inventory`.  In this code we use the MsPASS "create" method `save_inventory` to save a version of `Inventory` repackaged to mesh with MongoDB.

In [7]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
client=Client("IRIS")
starttime=UTCDateTime('2011-01-01T00:00:00.0')
endtime=UTCDateTime('2012-01-01T00:00:00.0')
inv=client.get_stations(network='TA',starttime=starttime,endtime=endtime,
                        format='xml',channel='BH?',level='response')
db.save_inventory(inv)

Database.save_inventory processing summary:
Number of site records processed= 653
number of site records saved= 653
number of channel records processed= 2091
number of channel records saved= 2079
(653, 2079, 653, 2091)


## Read
The R of CRUD is "Read" and is more-or-less the inverse of "create".   The keyword used for pulling "documents" from a MongoDB database, however, is `find`.  There are two basic methods in the core MongoDB API for fetching documents:  `find_one` and `find`.  They behave completely differently.

### find_one

Let's begin with a simple application of `find_one`.  As the name implies it always returns one and only one document.  Here is a default application to the "site" collection that was created under the hood when we ran `save_inventory` above:

In [47]:
doc = db.site.find_one()
print("The type of a document = ",type(doc))
print("This is the content of that document")
print(doc)

The type of a document =  <class 'dict'>
This is the content of that document
{'_id': ObjectId('659144c65870bd7b94bef62e'), 'loc': '', 'net': 'TA', 'sta': '034A', 'lat': 27.064699, 'lon': -98.683296, 'coords': [27.064699, -98.683296], 'elev': 0.155, 'edepth': 0.0, 'starttime': 1262908800.0, 'endtime': 1321574399.0, 'site_id': ObjectId('659144c65870bd7b94bef62e')}


As the output demonstrates a `find_one` returns data in a python dictionary.   You might also note the raw `print(doc)` output is a bit challenging to read.   For the rest of this tutorial we will use a construct I've used a lot that makes the output a bit easier to read.   I'll define a small little function we will use elsewhere in this tutorial to make output more readable.

In [49]:
from bson import json_util
def pretty_print(doc,indent=2):
    print(json_util.dumps(doc,indent=indent))
doc=db['site'].find_one()
pretty_print(doc)

{
  "_id": {
    "$oid": "659144c65870bd7b94bef62e"
  },
  "loc": "",
  "net": "TA",
  "sta": "034A",
  "lat": 27.064699,
  "lon": -98.683296,
  "coords": [
    27.064699,
    -98.683296
  ],
  "elev": 0.155,
  "edepth": 0.0,
  "starttime": 1262908800.0,
  "endtime": 1321574399.0,
  "site_id": {
    "$oid": "659144c65870bd7b94bef62e"
  }
}


Things of note in that box are:
1.  The `pretty_print` function definition is a bit trivial, which is why it isn't a standard MsPASS function.   It uses the `json_util.dumps` function to create the curly bracket formatted print that is a lot easier to understand than the raw dump of the python dictionary.   It shows more clearly that a document is always made of up of one or more key-value pairs.
2.  This example intentionally uses a variant of the syntax for interacting with the database handle.   Note in the first box I used `db.site` while in the second I used `db['site']`.   A powerful but confusing, in my opinion, feature of python is its capability to create that type of syntactic alternative incantation.   Technically, what it does is specify a "collection", which in this case is named "site".  In the jargon of MongoDB the `find` and `find_one` methods, which are the core MongoDB "read" methods, are "collection operation".   You should realize that `db` is the top-level symbol that refers to the "whole" database that is assumed to contain one more "collection"s.  The two incantations used above are alternative ways to get a handle to a specific "collection".   To clarify that point the following box illustrates a useful way to find the set of collections defined in our tutorial database at this point:

In [51]:
cursor=db.list_collections()
print("Current collections in tutorial database:")
for doc in cursor:
    print(doc['name'])

Current collections in tutorial database:
channel
site


### find
The above is also a good segway to the second standard MongoDB read method called `find`.  We defined the return of the `list_collection` function with the symbol "cursor".   That was a choice for the name, but consider this output:

In [52]:
print("Type type of the symbol cursor is ",type(cursor))

Type type of the symbol cursor is  <class 'pymongo.command_cursor.CommandCursor'>


A MongoDB `CommandCursor` is technically a __[forward iterator](https://www.boost.org/sgi/stl/ForwardIterator.html)__.   That means it acts like a list that can only be traversed "forward" with a construct like that above.   It is not at all the same thing, however, as a python list.   It is a handle that interacts with the database to sequentially return documents.   The following example with the `find` method illustrates the more common usage of a cursor:

A MongoDB `CommandCursor` is technically a  

In [54]:
cursor=db.site.find()
cursor.limit(3)
for doc in cursor:
    pretty_print(doc)

{
  "_id": {
    "$oid": "659144c65870bd7b94bef62e"
  },
  "loc": "",
  "net": "TA",
  "sta": "034A",
  "lat": 27.064699,
  "lon": -98.683296,
  "coords": [
    27.064699,
    -98.683296
  ],
  "elev": 0.155,
  "edepth": 0.0,
  "starttime": 1262908800.0,
  "endtime": 1321574399.0,
  "site_id": {
    "$oid": "659144c65870bd7b94bef62e"
  }
}
{
  "_id": {
    "$oid": "659144c65870bd7b94bef631"
  },
  "loc": "",
  "net": "TA",
  "sta": "035A",
  "lat": 26.937901,
  "lon": -98.102303,
  "coords": [
    26.937901,
    -98.102303
  ],
  "elev": 0.029,
  "edepth": 0.0,
  "starttime": 1263254400.0,
  "endtime": 1321315199.0,
  "site_id": {
    "$oid": "659144c65870bd7b94bef631"
  }
}
{
  "_id": {
    "$oid": "659144c65870bd7b94bef634"
  },
  "loc": "",
  "net": "TA",
  "sta": "035Z",
  "lat": 26.462999,
  "lon": -98.068298,
  "coords": [
    26.462999,
    -98.068298
  ],
  "elev": 0.019,
  "edepth": 0.0,
  "starttime": 1262995200.0,
  "endtime": 1321833599.0,
  "site_id": {
    "$oid": "659144

A few points of note about that simple 3 line code box:
1.  I used the default return for `find`.   The default returns "all", which in this would mean several hundred documents. For a large waveform data set it can easily be millions.
2.  To limit the output for this notebook I used a "method" of the `CommandCursor` class called "limit".  Here I did that with a separate line, but most python programmers would write the same expression as `cursor=db.site.find().limit(3)`.
3.  The output shows iterating through that (modified) cursor retrieves 3 documents from site.

Returning "all" is rarely what you want.  The more common use is to run `find` with a query as arg0 to the function.  The next subsection illustrates that use along with basics of the query language discussed in numerous printed sources, online sources, and the MsPASS User's Manual.   

### Mongo Query Language (MQL)
#### Single key match and basics
I will run a set of examples of increasing levels of complexity.   This particular section of this tutorial is intended as a hands on supplement to the section of the User's Manual titled "Using MongoDB with MsPASS" describing MQL.  

First, a unique match query:

In [62]:
query={'sta' : '134A'}
nsite=db.site.count_documents(query)
print("Number of site documents for station 134A=",nsite)
nchannel=db.channel.count_documents(query)
print("Number of channel documents for station 134A=",nchannel)
cursor=db.site.find(query)
for doc in cursor:
    pretty_print(doc)

Number of site documents for station 134A= 1
Number of channel documents for station 134A= 3
{
  "_id": {
    "$oid": "659144c65870bd7b94bef646"
  },
  "loc": "",
  "net": "TA",
  "sta": "134A",
  "lat": 32.572899,
  "lon": -98.079498,
  "coords": [
    32.572899,
    -98.079498
  ],
  "elev": 0.297,
  "edepth": 0.0,
  "starttime": 1258329600.0,
  "endtime": 1315526399.0,
  "site_id": {
    "$oid": "659144c65870bd7b94bef646"
  }
}


Notice:
1.  I used another important collection method called `count_documents` to fetch the expected number of documents the query would yield.  Standard practice in working through many queries is to do a check that the number it returns makes sense.
2.  We see there is one and only one station matching query is site and three in channel.  The reason channel has three, of course, is that there is a three-component sensor at that station that defines the recording channels.  To see why I didn't run the for loop over a cursor created from channel consider this:

In [64]:
# Note find_one accepts the same query but returns 
# only the first one it finds
doc=db.channel.find_one(query)
pretty_print(doc)

{
  "_id": {
    "$oid": "659144c65870bd7b94bef646"
  },
  "loc": "",
  "net": "TA",
  "sta": "134A",
  "lat": 32.572899,
  "lon": -98.079498,
  "coords": [
    32.572899,
    -98.079498
  ],
  "elev": 0.297,
  "edepth": 0.0,
  "starttime": 1258329600.0,
  "endtime": 1315487100.0,
  "chan": "BHE",
  "vang": 90.0,
  "hang": 90.7,
  "serialized_channel_data": {
    "$binary": {
      "base64": "gASVoBoAAAAAAACMHG9ic3B5LmNvcmUuaW52ZW50b3J5LmNoYW5uZWyUjAdDaGFubmVslJOUKYGUfZQojA5fbG9jYXRpb25fY29kZZSMAJSMCV9sYXRpdHVkZZSMGW9ic3B5LmNvcmUuaW52ZW50b3J5LnV0aWyUjAhMYXRpdHVkZZSTlEdAQElUwSJ0n4WUgZR9lCiMBWRhdHVtlE6MEWxvd2VyX3VuY2VydGFpbnR5lE6MEXVwcGVyX3VuY2VydGFpbnR5lE6MEm1lYXN1cmVtZW50X21ldGhvZJROdWKMCl9sb25naXR1ZGWUaAiMCUxvbmdpdHVkZZSTlEfAWIUWfseGPIWUgZR9lChoDk5oD05oEE5oEU51YowKX2VsZXZhdGlvbpRoCIwIRGlzdGFuY2WUk5RHQHKQAAAAAACFlIGUfZQoaA9OaBBOaBFOjAVfdW5pdJROdWKMBl9kZXB0aJRoGkcAAAAAAAAAAIWUgZR9lChoD05oEE5oEU5oHk51YowIX2F6aW11dGiUaAiMB0F6aW11dGiUk5RHQFaszMzMzM2FlIGUfZQoaA9OaBBOaBFOdWKMBF9kaXCUaAiMA0Rp

As you can see the attribute "serialized_channel_data" is huge and creates volumious output.   The reason is that it is a pickle format image of the raw "Inventory" record for that channel created by obspy's web service reader.  This example shows the common problem that documents can be too big to view with simple json_util dumps or a raw print.   For that reason it is often useful to specify a "projection" argument.   Here is an example where we extract an print only net, sta, chan, loc from each of the 3 channel documents:

In [67]:
projection={'net':1,'sta':1,'chan':1,'loc':1,'_id':0}
cursor=db.channel.find(query,projection)
for doc in cursor:
    print(doc)

{'loc': '', 'net': 'TA', 'sta': '134A', 'chan': 'BHE'}
{'loc': '', 'net': 'TA', 'sta': '134A', 'chan': 'BHN'}
{'loc': '', 'net': 'TA', 'sta': '134A', 'chan': 'BHZ'}


Here is a fancier variant using pandas to print a longer list of attributes in tabular form:

In [68]:
import pandas as pd
projection={
    'net':1,
    'sta':1,
    'chan':1,
    'lat':1,
    'lon':1,
    'elev':1,
    'hang':1,
    'vang':1,
    '_id':0,
}
cursor=db.channel.find(query,projection)
doclist=[]
for doc in cursor:
    doclist.append(doc)
df = pd.DataFrame.from_dict(doclist)
print(df)

  net   sta        lat        lon   elev chan  vang  hang
0  TA  134A  32.572899 -98.079498  0.297  BHE  90.0  90.7
1  TA  134A  32.572899 -98.079498  0.297  BHN  90.0   0.7
2  TA  134A  32.572899 -98.079498  0.297  BHZ   0.0   0.0


The pandas construct is useful enough let's create a function to make it easier to use it from now on.

In [72]:
import pandas as pd
def print_as_table(doclist):
    df = pd.DataFrame.from_dict(doclist)
    print(df)

#### Multiple key equality matching
Next let's do a query with multiple keys.   We will fetch the (shortened) record for the BHN component of a different station:

In [69]:
query={
    'sta' : '131A',
    'chan' : 'BHZ',
}
cursor=db.channel.find(query,projection)
for doc in cursor:
    pretty_print(doc)

{
  "net": "TA",
  "sta": "131A",
  "lat": 32.673698,
  "lon": -100.388802,
  "elev": 0.622,
  "chan": "BHZ",
  "vang": 0.0,
  "hang": 0.0
}


#### Range operator examples (compound query)
We often want to query by a range of values.  Here is an example that returns the coordinates of all TA stations with a 5 degree box defined by 30 to 35 latitude and -110 to -100 longitude: 

In [85]:
query={
    'lat' : {'$gte' : 30.0,'$lte' : 35.0},
    'lon' : {'$gte' : -110.0, '$lte' : -100},
}
projection={
   'net':1,
    'sta':1,
    'chan':1,
    'lat':1,
    'lon':1,
    'elev':1,
    '_id':0, 
}
cursor=db.site.find(query,projection)
doclist=[]
for doc in cursor:
    doclist.append(doc)
print_as_table(doclist)


   net   sta        lat         lon   elev
0   TA  121A  32.532398 -107.785103  1.652
1   TA  130A  32.596100 -100.965202  0.676
2   TA  131A  32.673698 -100.388802  0.622
3   TA  230A  31.887800 -101.112396  0.742
4   TA  231A  31.935301 -100.316299  0.574
5   TA  330A  31.406300 -101.175201  0.742
6   TA  331A  31.308500 -100.426598  0.615
7   TA  431A  30.682400 -100.607903  0.700
8   TA  530A  30.148899 -101.337898  0.636
9   TA  531A  30.164499 -100.546402  0.661
10  TA  MSTX  33.969601 -102.772400  1.167
11  TA  X30A  34.446098 -100.874001  0.698
12  TA  Y22D  34.073900 -106.921000  1.436
13  TA  Y22E  34.074200 -106.920799  1.444
14  TA  Y22E  34.074200 -106.920799  1.444
15  TA  Y30A  33.876598 -100.897797  0.812
16  TA  Y31A  33.962898 -100.261497  0.530
17  TA  Z30A  33.286098 -101.128197  0.729
18  TA  Z31A  33.318298 -100.143501  0.547


A variant using a regular expression:

In [93]:
query={
    'lat' : {'$gte' : 30.0,'$lte' : 35.0},
    'lon' : {'$gte' : -110.0, '$lte' : -100},
    'sta' : {'$regex' : 'Y.*'},
}
cursor=db.site.find(query,projection)
doclist=[]
for doc in cursor:
    doclist.append(doc)
print_as_table(doclist)

  net   sta        lat         lon   elev
0  TA  Y22D  34.073900 -106.921000  1.436
1  TA  Y22E  34.074200 -106.920799  1.444
2  TA  Y22E  34.074200 -106.920799  1.444
3  TA  Y30A  33.876598 -100.897797  0.812
4  TA  Y31A  33.962898 -100.261497  0.530


#### Geospatial query
MongoDB has some very useful geospatial query capabilities.  See the "MongoDB and MsPASS" section 
of the User's Manual for more about this capability.   
On the other hand, it is probably best thought of, at least at present, as an advanced feature.   The syntax is complex and, as noted in that section of the manual, MongoDB documentation is less than ideal and many online sources are inconsistent with the current implementation.  For this tutorial I will just show an example that is a variant of that shown in User's Manual page.

An IMPORTANT rule about using geospatial searches is that a special index is REQUIRED.  For this example the following is needed to make this work:

In [169]:
db.site.create_index({'location' : '2dsphere'})

'location_2dsphere'

Noting:
1.  'location' is the key used to tag the geoJSON format documents `save_inventory` created in the site collection.  It a constant tag in the MsPASS schema for these data.  Note also that if you were running this on the source collection the key has a different name of 'epicenter' since the content exactly matches the definition of the jargon term. 
2. '2dsphere' is a magic string that tells MongoDB to create a special index that uses spherical geometry for spatial calculations.  The alternative is '2d' but the alternative is not advised for most if not all seismology applications.  The '2d' index uses a map projection that produces distorted answers unless the area of study is small.  

Now that we have an index, we can do a search.  This search produces a similar result to the lat-lon range query above but for a circular (great circle path distance circle that is) region at the center of the same lat-lon box as above.  

In [173]:
query = {"location":{
        '$nearSphere': {
            '$geometry' : {
                'type' : 'Point',
                'coordinates' : [-105.0,32.5]
            },
            '$maxDistance' : 300000.0,
        }
      }
    }
# A flaw in the current MongoDB implementation is
# count_documents seems to not work with any geospatial 
# query.  If you remove this comment you will see 
# the error it throws.  If it works, it means MongoDB 
# developers fixed the problem
#n=db.site.count_documents(query)
cursor=db.site.find(query)
for doc in cursor:
    pretty_print(doc)

{
  "_id": {
    "$oid": "659144c95870bd7b94befdb7"
  },
  "loc": "",
  "net": "TA",
  "sta": "Y22D",
  "lat": 34.0739,
  "lon": -106.921,
  "coords": [
    34.0739,
    -106.921
  ],
  "elev": 1.436,
  "edepth": 0.0,
  "starttime": 1191024000.0,
  "endtime": 1575158399.9998999,
  "site_id": {
    "$oid": "659144c95870bd7b94befdb7"
  },
  "location": {
    "type": "Point",
    "coordinates": [
      -106.921,
      34.0739
    ]
  }
}
{
  "_id": {
    "$oid": "659144c95870bd7b94befdc3"
  },
  "loc": "01",
  "net": "TA",
  "sta": "Y22E",
  "lat": 34.0742,
  "lon": -106.920799,
  "coords": [
    34.0742,
    -106.920799
  ],
  "elev": 1.444,
  "edepth": 0.0,
  "starttime": 1301270400.0,
  "endtime": 1344297599.0,
  "site_id": {
    "$oid": "659144c95870bd7b94befdc3"
  },
  "location": {
    "type": "Point",
    "coordinates": [
      -106.920799,
      34.0742
    ]
  }
}
{
  "_id": {
    "$oid": "659144c95870bd7b94befdba"
  },
  "loc": "",
  "net": "TA",
  "sta": "Y22E",
  "lat": 34.074

Because of the pretty print of the full documents, that is a bit verbose, but it hopefully illustrates the point.  Although geospatial queries are complex, they have a lot of potential use for workflows that need to group data by the spatial location of stations (a "virtual array" concept) or by source (stacking of closely spaced sources).  

### Sorting
There are many situations where it is advantageous to 
sort the return of a query by one or more keys.   Sorting is technically a method of the "Cursor" object returned by a query but more magic happens when the 
client passes the query to the MongoDB server to assure 
the operation is done efficiently.   The reason I point 
that out here is mostly to clarify the what a sort clause 
appears in typical usage.  The User Manual addresses this 
in more detail, but here is an example that sorts 
channel documents to a form sensible for miniseed that 
uses the net:sta:chan:loc:time-interval as a unique 
key combination.  

In [176]:
# this is a test to verify sort syntax - delete when completed
filter_clause = {
    "_id":0,
    "sta":1,
    "chan":1,
    "starttime":1,
    "endtime":1,
}
sort_clause = [
    ("net",1),
    ("sta",1),
    ("chan",1),
    ("starttime",1),
  ]
cursor=db.channel.find({},filter_clause).sort(sort_clause).limit(6)
doclist=[]
for doc in cursor:
    doclist.append(doc)
from obspy import UTCDateTime
for doc in doclist:
    doc['starttime']=UTCDateTime(doc['starttime'])
    doc['endtime']=UTCDateTime(doc['endtime'])
print_as_table(doclist)
    

    sta                    starttime                      endtime chan
0  034A  2010-01-08T00:00:00.000000Z  2011-11-17T17:05:00.000000Z  BHE
1  034A  2010-01-08T00:00:00.000000Z  2011-11-17T17:05:00.000000Z  BHN
2  034A  2010-01-08T00:00:00.000000Z  2011-11-17T17:05:00.000000Z  BHZ
3  035A  2010-01-12T00:00:00.000000Z  2011-11-14T17:40:00.000000Z  BHE
4  035A  2010-01-12T00:00:00.000000Z  2011-11-14T17:40:00.000000Z  BHN
5  035A  2010-01-12T00:00:00.000000Z  2011-11-14T17:40:00.000000Z  BHZ


Noting:
1.  The "sort" function call appears after the find function with arguments.   That is the syntax because "sort" is a Cursor "method".
2.  I added a second qualifier, limit, to only return the first 6 documents.  I did that just to keep the volume oof output under control.   The number return is much larger if you remove the `.limit(6)` qualifier.
3.  I did a projection and used the `print_as_table` function we defined to make a more readable report. 

## Update

## Delete